In [2]:
!pip install pandas
!pip install datasets
!pip install pytorch-ignite
!pip install torchmetrics
!pip install pytorch-lightning
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 847.9 kB/s eta 0:00:00a 0:00:01
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 1.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [12]:
import ast
import pandas as pd
from eval_utils import rougel_eval, selfbleu_eval, bertscore_eval, bleurt_eval, concat_gen, tokenize
from gen_answerability import run_generate

Seed set to 42


In [17]:
TYPE2CONTROL_SIGNAL = ['What', 'How', 'Who', 'Why', 'Where', 'When', 'Which']

# path for generated result
result_dir = 'output/'
result_df = pd.read_csv(result_dir)
for i in TYPE2CONTROL_SIGNAL:
    result_df[i] = result_df[i].apply(lambda x: ast.literal_eval(x))
    
# path for golden data
test_dir = 'data/'
gold_df = pd.read_csv(test_dir)
gold_df = gold_df.loc[:,['cor_section', 'question']]
gold_df.columns = ['context', 'question']

# merge dataframe
gb = gold_df.groupby(['context'])
result = pd.DataFrame(gb['question'].unique())

tot_df = pd.merge(result, result_df, on="context")
tot_df = tot_df.dropna()
tot_df['tot_gen'] = tot_df.apply(lambda x: concat_gen(x, 4, TYPE2CONTROL_SIGNAL), axis=1)

EmptyDataError: No columns to parse from file

In [ ]:
rougel_eval(tot_df)

In [ ]:
selfbleu_eval(tot_df)

In [ ]:
bertscore_eval(tot_df)

In [ ]:
bleurt_eval(tot_df)

In [ ]:
# answerability evaluation model checkpoint
ckpt_path = 'checkpoint/epoch=5-step=1109.ckpt'
    
ans_df = tot_df.loc[:, ['context', 'tot_gen']]
ans_df = ans_df.explode('tot_gen').reset_index(inplace=False, drop=True)

run_generate(ans_df, ckpt_path)